### Converting Poses to poses_bound.npy

1. Make sure your poses are in camera-to-world format, not world-to-camera.
2. Make sure your rotation matrices have the columns in the correct coordinate frame [down, right, backwards].
3. Concatenate each pose with the [height, width, focal] intrinsics vector to get a 3x5 matrix.
4. Flatten each of those into 15 elements and concatenate the close and far depths.
5. Stack the 17-d vectors to get a Nx17 matrix and use np.save to store it as poses_bounds.npy in the scene's base directory (same level containing the images/ directory).

In [ ]:
from scipy.io import loadmat
from numpy.linalg import inv
import numpy as np
poses = loadmat('Calib_Results.mat')
print(poses)


In [ ]:
KK = poses['KK']
print(KK)

### Step 1

In [ ]:
Rc_1 = poses['Rc_1']
print(Rc_1)

In [ ]:
Tc_1 = poses['Tc_1']
print(Tc_1)

In [ ]:
T_w2c_1 = Rc_1
T_w2c_1 = np.append(T_w2c_1, Tc_1, axis = 1)

T_w2c_1 = np.append(T_w2c_1, np.transpose([[0],[0],[0],[1]]), axis = 0)
print(T_w2c_1)

T_c2w_1 = inv(T_w2c_1)
print('Inverse')
print(T_c2w_1)

T_c2w_1 = T_c2w_1[0:3,:]
print('Final....')
print(T_c2w_1)

### Step 2

In [ ]:
rot_mat = np.array([[0, -1, 0, 0],
           [1, 0, 0, 0],
           [0, 0, 1, 0],
           [0, 0, 0, 1]])
rot_T_c2w_1 = np.matmul(T_c2w_1, rot_mat)
print('Final rotated matrix....')
print(rot_T_c2w_1)

### Step 3

In [ ]:
# [height, width, focal]

focal = (KK[0][0]+KK[1][1])/2.0
print('Focal')
print(focal)

height = KK[1][2]*2.0
print('Height')
print(height)

width = KK[0][2]*2.0
print('Width')
print(width)

intrinsics = [height, width, focal]
print('Intrinsics')
print(intrinsics)

In [ ]:
rot_T_c2w_1 = rot_T_c2w_1.flatten()
print('Flatten')
print(rot_T_c2w_1)

poses_bound = np.append(rot_T_c2w_1, intrinsics)
print('Poses mat unfinished...')
print(poses_bound)

### Step 4

In [ ]:
from plyfile import PlyData, PlyElement

mesh = PlyData.read('mesh_Gt.ply')
print(mesh)

In [ ]:
[F, V] = PlyData.read('mesh_Gt.ply')
print(V)

In [ ]:
print(F)

In [ ]:
x = np.memmap.tolist(mesh['vertex']['x'])
y = np.memmap.tolist(mesh['vertex']['y'])
z = np.memmap.tolist(mesh['vertex']['z'])


In [ ]:
print(x[0],' ',y[0],' ',z[0])


In [ ]:
V = np.column_stack((x, y, z))
print(V)

add = np.ones(1002858)
V = np.column_stack((V, add))
print(V)

In [ ]:
V_c = np.matmul(T_w2c_1, np.transpose(V))
print(V_c)

In [ ]:
minz = min(V_c[2])
maxz = max(V_c[2])
print(minz)
print(maxz)

In [ ]:
poses_bound = np.append(poses_bound, [minz, maxz])
print('Poses matix final...')
print(poses_bound)

### Step 5

In [ ]:
# save to npy file
np.save('poses_bound.npy', poses_bound)


In [ ]:
np.load('poses_bound.npy')

## Reshuffling to match NERF order

In [9]:
from scipy.io import loadmat
from numpy.linalg import inv
import numpy as np
poses = loadmat('Calib_Results.mat')
print(poses)

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Nov 09 17:02:54 2017', '__version__': '1.0', '__globals__': [], 'KK': array([[3.77207747e+03, 0.00000000e+00, 3.05875000e+02],
       [0.00000000e+00, 3.75900543e+03, 2.55875000e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]), 'Rc_1': array([[-0.02365614,  0.99963052, -0.01338513],
       [ 0.45312528, -0.00121338, -0.89144604],
       [-0.891133  , -0.027153  , -0.452929  ]]), 'Tc_1': array([[ -81.42967876],
       [ -14.67307917],
       [1623.334915  ]]), 'Rc_2': array([[-0.28771447,  0.95763136, -0.01275215],
       [ 0.4368058 ,  0.11936278, -0.8916015 ],
       [-0.852303  , -0.262097  , -0.452641  ]]), 'Tc_2': array([[ -57.55750749],
       [ -23.56542844],
       [1639.708698  ]]), 'Rc_3': array([[-0.57504278,  0.81803644, -0.01192571],
       [ 0.37664138,  0.25176541, -0.89149059],
       [-0.726269  , -0.517137  , -0.452882  ]]), 'Tc_3': array([[ -23.66517847],
       [ -30.07418965],
  

In [10]:
KK = poses['KK']
print(KK)

[[3.77207747e+03 0.00000000e+00 3.05875000e+02]
 [0.00000000e+00 3.75900543e+03 2.55875000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [11]:
Rc_1 = poses['Rc_1']
print(Rc_1)

[[-0.02365614  0.99963052 -0.01338513]
 [ 0.45312528 -0.00121338 -0.89144604]
 [-0.891133   -0.027153   -0.452929  ]]


In [12]:
Tc_1 = poses['Tc_1']
print(Tc_1)
print(Tc_1.shape)

[[ -81.42967876]
 [ -14.67307917]
 [1623.334915  ]]
(3, 1)


In [13]:
T_w2c_1 = Rc_1
T_w2c_1 = np.append(T_w2c_1, Tc_1, axis = 1)
print(T_w2c_1)

[[-2.36561422e-02  9.99630519e-01 -1.33851320e-02 -8.14296788e+01]
 [ 4.53125275e-01 -1.21338236e-03 -8.91446037e-01 -1.46730792e+01]
 [-8.91133000e-01 -2.71530000e-02 -4.52929000e-01  1.62333491e+03]]


In [14]:
#WORLD to CAMERA ---> CAMERA to WORLD

T_w2c_1 = np.append(T_w2c_1, np.transpose([[0],[0],[0],[1]]), axis = 0)
print(T_w2c_1)

T_c2w_1 = inv(T_w2c_1)
print('Inverse')
print(T_c2w_1)

T_c2w_1 = T_c2w_1[0:3,:]
print('Final....')
print(T_c2w_1)


[[-2.36561422e-02  9.99630519e-01 -1.33851320e-02 -8.14296788e+01]
 [ 4.53125275e-01 -1.21338236e-03 -8.91446037e-01 -1.46730792e+01]
 [-8.91133000e-01 -2.71530000e-02 -4.52929000e-01  1.62333491e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Inverse
[[-2.36558586e-02  4.53125107e-01 -8.91132923e-01  1.45132964e+03]
 [ 9.99630578e-01 -1.21338002e-03 -2.71533164e-02  1.25460719e+02]
 [-1.33849959e-02 -8.91446096e-01 -4.52929159e-01  7.21085522e+02]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Final....
[[-2.36558586e-02  4.53125107e-01 -8.91132923e-01  1.45132964e+03]
 [ 9.99630578e-01 -1.21338002e-03 -2.71533164e-02  1.25460719e+02]
 [-1.33849959e-02 -8.91446096e-01 -4.52929159e-01  7.21085522e+02]]


In [15]:
# Intrinsics [height, width, focal]

focal = (KK[0][0]+KK[1][1])/2.0

height = 512.0

width = 612.0

intrinsics = np.array([height,width,focal])
intrinsics = intrinsics.reshape(3,1)
print('Intrinsics')
print(intrinsics)


Intrinsics
[[ 512.        ]
 [ 612.        ]
 [3765.54145104]]


In [16]:
T_c2w_1 = np.append(T_c2w_1, intrinsics, axis = 1)
print(T_c2w_1)



[[-2.36558586e-02  4.53125107e-01 -8.91132923e-01  1.45132964e+03
   5.12000000e+02]
 [ 9.99630578e-01 -1.21338002e-03 -2.71533164e-02  1.25460719e+02
   6.12000000e+02]
 [-1.33849959e-02 -8.91446096e-01 -4.52929159e-01  7.21085522e+02
   3.76554145e+03]]


In [17]:
# Flatten

T_c2w_1 = T_c2w_1.flatten()
print('Flatten')
print(T_c2w_1)


Flatten
[-2.36558586e-02  4.53125107e-01 -8.91132923e-01  1.45132964e+03
  5.12000000e+02  9.99630578e-01 -1.21338002e-03 -2.71533164e-02
  1.25460719e+02  6.12000000e+02 -1.33849959e-02 -8.91446096e-01
 -4.52929159e-01  7.21085522e+02  3.76554145e+03]


In [18]:
# z near-far values

from plyfile import PlyData, PlyElement

mesh = PlyData.read('mesh_Gt.ply')
print(mesh)

x = np.memmap.tolist(mesh['vertex']['x'])
y = np.memmap.tolist(mesh['vertex']['y'])
z = np.memmap.tolist(mesh['vertex']['z'])

print(x[0],' ',y[0],' ',z[0])


ply
format binary_little_endian 1.0
comment VCGLIB generated
element vertex 1002858
property float x
property float y
property float z
property float nx
property float ny
property float nz
element face 2005728
property list uchar int vertex_indices
end_header
49.71357345581055   84.95451354980469   2.3159995079040527


In [19]:
V = np.column_stack((x, y, z))

add = np.ones(1002858)
V = np.column_stack((V, add))
print('V mat')
print(V)

V_c = np.matmul(T_w2c_1, np.transpose(V))
print('V_c mat')
print(V_c)



V mat
[[4.97135735e+01 8.49545135e+01 2.31599951e+00 1.00000000e+00]
 [4.97125397e+01 8.53960114e+01 2.31780457e+00 1.00000000e+00]
 [4.97161560e+01 8.55068207e+01 2.20618343e+00 1.00000000e+00]
 ...
 [8.05606613e+01 1.04334747e+02 3.13488808e+01 1.00000000e+00]
 [8.05999069e+01 1.04359329e+02 3.15576973e+01 1.00000000e+00]
 [5.39716748e-05 5.27609475e-02 1.86067828e-09 1.00000000e+00]]
V_c mat
[[ 2.28641437e+00  2.72774934e+00  2.83992624e+00 ...  2.05411555e+01
   2.05620048e+01 -8.13769386e+01]
 [ 5.68572659e+00  5.68311334e+00  5.78412176e+00 ... -6.24144080e+00
  -6.40983612e+00 -1.46731187e+01]
 [ 1.57567776e+03  1.57566587e+03  1.57571020e+03 ...  1.53451283e+03
   1.53438261e+03  1.62333343e+03]
 [ 1.00000000e+00  1.00000000e+00  1.00000000e+00 ...  1.00000000e+00
   1.00000000e+00  1.00000000e+00]]


In [20]:
minz = min(V_c[2,:])
maxz = max(V_c[2])
print(minz)
print(maxz)

1498.930845036865
1623.3334342852093


In [21]:
poses_bounds_1 = np.append(T_c2w_1, [minz, maxz])
print('Poses matix final...')
print(poses_bounds_1)

Poses matix final...
[-2.36558586e-02  4.53125107e-01 -8.91132923e-01  1.45132964e+03
  5.12000000e+02  9.99630578e-01 -1.21338002e-03 -2.71533164e-02
  1.25460719e+02  6.12000000e+02 -1.33849959e-02 -8.91446096e-01
 -4.52929159e-01  7.21085522e+02  3.76554145e+03  1.49893085e+03
  1.62333343e+03]


## Making it work for 20 images

In [57]:
from scipy.io import loadmat
from numpy.linalg import inv
import numpy as np
poses = loadmat('Calib_Results.mat')
print(poses)


{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Thu Nov 09 17:02:54 2017', '__version__': '1.0', '__globals__': [], 'KK': array([[3.77207747e+03, 0.00000000e+00, 3.05875000e+02],
       [0.00000000e+00, 3.75900543e+03, 2.55875000e+02],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]]), 'Rc_1': array([[-0.02365614,  0.99963052, -0.01338513],
       [ 0.45312528, -0.00121338, -0.89144604],
       [-0.891133  , -0.027153  , -0.452929  ]]), 'Tc_1': array([[ -81.42967876],
       [ -14.67307917],
       [1623.334915  ]]), 'Rc_2': array([[-0.28771447,  0.95763136, -0.01275215],
       [ 0.4368058 ,  0.11936278, -0.8916015 ],
       [-0.852303  , -0.262097  , -0.452641  ]]), 'Tc_2': array([[ -57.55750749],
       [ -23.56542844],
       [1639.708698  ]]), 'Rc_3': array([[-0.57504278,  0.81803644, -0.01192571],
       [ 0.37664138,  0.25176541, -0.89149059],
       [-0.726269  , -0.517137  , -0.452882  ]]), 'Tc_3': array([[ -23.66517847],
       [ -30.07418965],
  

In [58]:
KK = poses['KK']
print(KK)


[[3.77207747e+03 0.00000000e+00 3.05875000e+02]
 [0.00000000e+00 3.75900543e+03 2.55875000e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]


In [59]:
from plyfile import PlyData, PlyElement

# # rotation matrix
# rot_mat = np.array([[0, -1, 0, 0],
#                    [1, 0, 0, 0],
#                    [0, 0, 1, 0],
#                    [0, 0, 0, 1]])

# [height, width, focal]
focal = (KK[0][0]+KK[1][1])/2.0

height = 512.0

width = 612.0

intrinsics = np.array([height,width,focal])
intrinsics = intrinsics.reshape(3,1)
print('Intrinsics')
print(intrinsics)

#Verticies
mesh = PlyData.read('mesh_Gt.ply')

x = np.memmap.tolist(mesh['vertex']['x'])
y = np.memmap.tolist(mesh['vertex']['y'])
z = np.memmap.tolist(mesh['vertex']['z'])

V = np.column_stack((x, y, z))
add = np.ones(1002858)
V = np.column_stack((V, add))
print('Verticies')
print(V)

Intrinsics
[[ 512.        ]
 [ 612.        ]
 [3765.54145104]]
Verticies
[[4.97135735e+01 8.49545135e+01 2.31599951e+00 1.00000000e+00]
 [4.97125397e+01 8.53960114e+01 2.31780457e+00 1.00000000e+00]
 [4.97161560e+01 8.55068207e+01 2.20618343e+00 1.00000000e+00]
 ...
 [8.05606613e+01 1.04334747e+02 3.13488808e+01 1.00000000e+00]
 [8.05999069e+01 1.04359329e+02 3.15576973e+01 1.00000000e+00]
 [5.39716748e-05 5.27609475e-02 1.86067828e-09 1.00000000e+00]]


## Without rotation

In [6]:
poses_bounds = []

for i in range(1,21):
    
    #Step 1
    
    Rc = poses['Rc_'+str(i)]
    Tc = poses['Tc_'+str(i)]
    
    T_w2c = Rc
    T_w2c = np.append(T_w2c, Tc, axis = 1)

    T_w2c = np.append(T_w2c, np.transpose([[0],[0],[0],[1]]), axis = 0)
#     print(T_w2c)

    T_c2w = inv(T_w2c)
#     print('Inverse')
#     print(T_c2w)

    T_c2w = T_c2w[0:3,:]
#     print('Final....')
#     print(T_c2w)

    T_c2w = np.append(T_c2w, intrinsics, axis = 1)
#     print(T_c2w)

    T_c2w = T_c2w.flatten()
#     print('Flatten')
#     print(T_c2w_1)
    
    
    V_c = np.matmul(T_w2c, np.transpose(V))
#     print('Camera Verticies')
#     print(V_c)
    minz = min(V_c[2])
    maxz = max(V_c[2])
    
    poses_bound_img = np.append(T_c2w, [minz, maxz])
    poses_bound_img.tolist()
#     print('Poses matix final...')
#     print(poses_bound_img)
    
    poses_bounds.append(poses_bound_img)
    
print('FINAL')
print(poses_bounds)



FINAL
[array([-2.36558586e-02,  4.53125107e-01, -8.91132923e-01,  1.45132964e+03,
        5.12000000e+02,  9.99630578e-01, -1.21338002e-03, -2.71533164e-02,
        1.25460719e+02,  6.12000000e+02, -1.33849959e-02, -8.91446096e-01,
       -4.52929159e-01,  7.21085522e+02,  3.76554145e+03,  1.49893085e+03,
        1.62333343e+03]), array([-2.87714685e-01,  4.36805694e-01, -8.52303773e-01,  1.39126328e+03,
        5.12000000e+02,  9.57631241e-01,  1.19362714e-01, -2.62096973e-01,
        4.87694388e+02,  6.12000000e+02, -1.27522396e-02, -8.91601541e-01,
       -4.52641513e-01,  7.20455266e+02,  3.76554145e+03,  1.49826678e+03,
        1.63969482e+03]), array([-5.75042933e-01,  3.76641297e-01, -7.26269487e-01,  1.19739118e+03,
        5.12000000e+02,  8.18036282e-01,  2.51765315e-01, -5.17137066e-01,
        8.81152254e+02,  6.12000000e+02, -1.19257837e-02, -8.91490627e-01,
       -4.52882374e-01,  7.20990838e+02,  3.76554145e+03,  1.49593067e+03,
        1.65180098e+03]), array([-8.07314

## With Rotation

wt_rot1 = original
wt_rot2 = no abs
wt_rot3 = only rx_180 and no abs
wt_rot4 = only rz_90 and no abs
wt_rot5 = only rx_180 and abs //worked for first image
wt_rot6 = only rz_90 and abs
wt_rot7 = reverse both matmul
wt_rot7 = reverse rz_90 //worked for 1st image
wt_rot8 = reverse rx_180 //worked for 1st image
wt_rot9 = reverse V 

In [54]:
poses_bounds = []

# rotation matrix
rz_90 = np.array([[0, -1, 0, 0],
                   [1, 0, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])


rx_180 =np.array([[1, 0, 0, 0],
                 [0, -1, 0, 0],
                 [0, 0, -1, 0],
                 [0, 0, 0, 1]])


for i in range(1,21):
    
    #Step 1
    
    Rc = poses['Rc_'+str(i)]
    Tc = poses['Tc_'+str(i)]
    
    T_w2c = Rc
    T_w2c = np.append(T_w2c, Tc, axis = 1)

    T_w2c = np.append(T_w2c, np.transpose([[0],[0],[0],[1]]), axis = 0)
#     print(T_w2c)

    T_w2c_rot_old = np.matmul(rx_180, T_w2c)
    T_w2c_rot = np.matmul(rz_90, T_w2c_rot_old)   
    
    
#     print('Final rotated matrix....')
#     print(T_w2c_rot)


    T_c2w_rot = inv(T_w2c_rot)
#     print('Inverse')
#     print(T_c2w)


    T_c2w_rot = T_c2w_rot[0:3,:]
#     print('Final....')
#     print(T_c2w)

    T_c2w_rot = np.append(T_c2w_rot, intrinsics, axis = 1)
#     print(T_c2w)

    T_c2w_rot = T_c2w_rot.flatten()
#     print('Flatten')
#     print(T_c2w_1)
    
    
    V_c = np.matmul(T_w2c_rot, np.transpose(V))
    
#     print('Camera Verticies')
#     print(V_c)
    minz = min(np.abs(V_c[2]))
    maxz = max(np.abs(V_c[2]))
    
#     print(minz)
#     print(maxz)
    
    poses_bound_img = np.append(T_c2w_rot, [minz, maxz])
    poses_bound_img.tolist()
#     print('Poses matix final...')
#     print(poses_bound_img)
    
    poses_bounds.append(poses_bound_img)
    
print('FINAL')
print(poses_bounds)

FINAL
[array([ 4.53125107e-01, -2.36558586e-02,  8.91132923e-01,  1.45132964e+03,
        5.12000000e+02, -1.21338002e-03,  9.99630578e-01,  2.71533164e-02,
        1.25460719e+02,  6.12000000e+02, -8.91446096e-01, -1.33849959e-02,
        4.52929159e-01,  7.21085522e+02,  3.76554145e+03,  2.24708882e+01,
        1.12726566e+04]), array([ 4.36805694e-01, -2.87714685e-01,  8.52303773e-01,  1.39126328e+03,
        5.12000000e+02,  1.19362714e-01,  9.57631241e-01,  2.62096973e-01,
        4.87694388e+02,  6.12000000e+02, -8.91601541e-01, -1.27522396e-02,
        4.52641513e-01,  7.20455266e+02,  3.76554145e+03,  1.00490739e+00,
        9.70964015e+03]), array([ 3.76641297e-01, -5.75042933e-01,  7.26269487e-01,  1.19739118e+03,
        5.12000000e+02,  2.51765315e-01,  8.18036282e-01,  5.17137066e-01,
        8.81152254e+02,  6.12000000e+02, -8.91490627e-01, -1.19257837e-02,
        4.52882374e-01,  7.20990838e+02,  3.76554145e+03,  2.88426353e+01,
        7.16194351e+03]), array([ 2.76005

In [55]:
# save to npy file
np.save('poses_bounds.npy', poses_bounds)


In [56]:
np.load('poses_bounds.npy')


array([[ 4.53125107e-01, -2.36558586e-02,  8.91132923e-01,
         1.45132964e+03,  5.12000000e+02, -1.21338002e-03,
         9.99630578e-01,  2.71533164e-02,  1.25460719e+02,
         6.12000000e+02, -8.91446096e-01, -1.33849959e-02,
         4.52929159e-01,  7.21085522e+02,  3.76554145e+03,
         2.24708882e+01,  1.12726566e+04],
       [ 4.36805694e-01, -2.87714685e-01,  8.52303773e-01,
         1.39126328e+03,  5.12000000e+02,  1.19362714e-01,
         9.57631241e-01,  2.62096973e-01,  4.87694388e+02,
         6.12000000e+02, -8.91601541e-01, -1.27522396e-02,
         4.52641513e-01,  7.20455266e+02,  3.76554145e+03,
         1.00490739e+00,  9.70964015e+03],
       [ 3.76641297e-01, -5.75042933e-01,  7.26269487e-01,
         1.19739118e+03,  5.12000000e+02,  2.51765315e-01,
         8.18036282e-01,  5.17137066e-01,  8.81152254e+02,
         6.12000000e+02, -8.91490627e-01, -1.19257837e-02,
         4.52882374e-01,  7.20990838e+02,  3.76554145e+03,
         2.88426353e+01,  7.1

## Only with images 1,2,3,4,5,17,18,19,20

In [63]:
poses_bounds = []

# rotation matrix
rz_90 = np.array([[0, -1, 0, 0],
                   [1, 0, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])


rx_180 =np.array([[1, 0, 0, 0],
                 [0, -1, 0, 0],
                 [0, 0, -1, 0],
                 [0, 0, 0, 1]])


for i in range(1,6):
    
    #Step 1
    
    Rc = poses['Rc_'+str(i)]
    Tc = poses['Tc_'+str(i)]
    
    T_w2c = Rc
    T_w2c = np.append(T_w2c, Tc, axis = 1)

    T_w2c = np.append(T_w2c, np.transpose([[0],[0],[0],[1]]), axis = 0)

    T_w2c_rot_old = np.matmul(rx_180, T_w2c)
    T_w2c_rot = np.matmul(rz_90, T_w2c_rot_old)   

    T_c2w_rot = inv(T_w2c_rot)

    T_c2w_rot = T_c2w_rot[0:3,:]

    T_c2w_rot = np.append(T_c2w_rot, intrinsics, axis = 1)

    T_c2w_rot = T_c2w_rot.flatten()   
    
    V_c = np.matmul(T_w2c_rot, np.transpose(V))
    minz = min(np.abs(V_c[2]))
    maxz = max(np.abs(V_c[2]))
    
    poses_bound_img = np.append(T_c2w_rot, [minz, maxz])
    poses_bound_img.tolist()
    
    poses_bounds.append(poses_bound_img)
    
print('FINAL')
print(poses_bounds)

FINAL
[array([ 4.53125107e-01, -2.36558586e-02,  8.91132923e-01,  1.45132964e+03,
        5.12000000e+02, -1.21338002e-03,  9.99630578e-01,  2.71533164e-02,
        1.25460719e+02,  6.12000000e+02, -8.91446096e-01, -1.33849959e-02,
        4.52929159e-01,  7.21085522e+02,  3.76554145e+03,  1.49893085e+03,
        1.62333343e+03]), array([ 4.36805694e-01, -2.87714685e-01,  8.52303773e-01,  1.39126328e+03,
        5.12000000e+02,  1.19362714e-01,  9.57631241e-01,  2.62096973e-01,
        4.87694388e+02,  6.12000000e+02, -8.91601541e-01, -1.27522396e-02,
        4.52641513e-01,  7.20455266e+02,  3.76554145e+03,  1.49826678e+03,
        1.63969482e+03]), array([ 3.76641297e-01, -5.75042933e-01,  7.26269487e-01,  1.19739118e+03,
        5.12000000e+02,  2.51765315e-01,  8.18036282e-01,  5.17137066e-01,
        8.81152254e+02,  6.12000000e+02, -8.91490627e-01, -1.19257837e-02,
        4.52882374e-01,  7.20990838e+02,  3.76554145e+03,  1.49593067e+03,
        1.65180098e+03]), array([ 2.76005

In [64]:
# save to npy file
np.save('poses_bounds.npy', poses_bounds)


In [65]:
np.load('poses_bounds.npy')


array([[ 4.53125107e-01, -2.36558586e-02,  8.91132923e-01,
         1.45132964e+03,  5.12000000e+02, -1.21338002e-03,
         9.99630578e-01,  2.71533164e-02,  1.25460719e+02,
         6.12000000e+02, -8.91446096e-01, -1.33849959e-02,
         4.52929159e-01,  7.21085522e+02,  3.76554145e+03,
         1.49893085e+03,  1.62333343e+03],
       [ 4.36805694e-01, -2.87714685e-01,  8.52303773e-01,
         1.39126328e+03,  5.12000000e+02,  1.19362714e-01,
         9.57631241e-01,  2.62096973e-01,  4.87694388e+02,
         6.12000000e+02, -8.91601541e-01, -1.27522396e-02,
         4.52641513e-01,  7.20455266e+02,  3.76554145e+03,
         1.49826678e+03,  1.63969482e+03],
       [ 3.76641297e-01, -5.75042933e-01,  7.26269487e-01,
         1.19739118e+03,  5.12000000e+02,  2.51765315e-01,
         8.18036282e-01,  5.17137066e-01,  8.81152254e+02,
         6.12000000e+02, -8.91490627e-01, -1.19257837e-02,
         4.52882374e-01,  7.20990838e+02,  3.76554145e+03,
         1.49593067e+03,  1.6

## Making it work for each image

In [ ]:
from scipy.io import loadmat
from numpy.linalg import inv
import numpy as np
poses = loadmat('Calib_Results.mat')
print(poses)


In [ ]:
KK = poses['KK']
print(KK)

In [ ]:
from plyfile import PlyData, PlyElement

# rotation matrix
rot_mat = np.array([[0, -1, 0, 0],
                   [1, 0, 0, 0],
                   [0, 0, 1, 0],
                   [0, 0, 0, 1]])

# [height, width, focal]
focal = (KK[0][0]+KK[1][1])/2.0

height = 512.0

width = 612.0

intrinsics = [height, width, focal]
print('Intrinsics')
print(intrinsics)

#Verticies
mesh = PlyData.read('mesh_Gt.ply')

x = np.memmap.tolist(mesh['vertex']['x'])
y = np.memmap.tolist(mesh['vertex']['y'])
z = np.memmap.tolist(mesh['vertex']['z'])

V = np.column_stack((x, y, z))
add = np.ones(1002858)
V = np.column_stack((V, add))
print('Verticies')
print(V)


### Test 1: Rot mat before inversion

In [ ]:
poses_bound = []

for i in range(1,21):
    
    #Step 1
    
    Rc = poses['Rc_'+str(i)]
    Tc = poses['Tc_'+str(i)]
    
    T_w2c = Rc
    T_w2c = np.append(T_w2c, Tc, axis = 1)

    T_w2c = np.append(T_w2c, np.transpose([[0],[0],[0],[1]]), axis = 0)
#     print(T_w2c)
    
    #Step 2
    
    rot_T_w2c = np.matmul(T_w2c, rot_mat)
#     print('Rotated matrix....')
#     print(rot_T_w2c)
    
    #Continue Step 1

    rot_T_c2w = inv(rot_T_w2c)
#     print('Inverse')
#     print(rot_T_c2w)

    rot_T_c2w = rot_T_c2w[0:3,:]
#     print('Final....')
#     print(rot_T_c2w)
    
    #Step 3
    
    rot_T_c2w = rot_T_c2w.flatten()
#     print('Flatten')
#     print(rot_T_c2w)

    poses_bound_img = np.append(rot_T_c2w, intrinsics)
#     print('Poses mat unfinished...')
#     print(poses_bound_img)
    
    #Step 4
    
    V_c = np.matmul(rot_T_w2c, np.transpose(V))
#     print('Camera Verticies')
#     print(V_c)
    minz = min(V_c[2])
    maxz = max(V_c[2])
    
    poses_bound_img = np.append(poses_bound_img, [minz, maxz])
    poses_bound_img.tolist()
#     print('Poses matix final...')
#     print(poses_bound_img)
    
    poses_bound.append(poses_bound_img)
    
print('FINAL')
print(poses_bound)


### Test 2: Rot mat after inversion 

In [ ]:
poses_bound = []

for i in range(1,21):
    
    #Step 1
    
    Rc = poses['Rc_'+str(i)]
    Tc = poses['Tc_'+str(i)]
    
    T_w2c = Rc
    T_w2c = np.append(T_w2c, Tc, axis = 1)

    T_w2c = np.append(T_w2c, np.transpose([[0],[0],[0],[1]]), axis = 0)
#     print(T_w2c)
    
    T_c2w = inv(T_w2c)
#     print('Inverse')
#     print(T_c2w)

    T_c2w = T_c2w[0:3,:]
#     print('Final....')
#     print(T_c2w)
    
    #Step 2
    
    rot_T_c2w = np.matmul(T_c2w, rot_mat)
#     print('Final rotated matrix....')
#     print(rot_T_c2w)
    
    #Step 3
    
    rot_T_c2w = rot_T_c2w.flatten()
#     print('Flatten')
#     print(rot_T_c2w)

    poses_bound_img = np.append(rot_T_c2w, intrinsics)
#     print('Poses mat unfinished...')
#     print(poses_bound)
    
    #Step 4
    
    V_c = np.matmul(T_w2c, np.transpose(V))
#     print('Camera Verticies')
#     print(V_c)
    minz = min(V_c[2])
    maxz = max(V_c[2])
    
    poses_bound_img = np.append(poses_bound_img, [minz, maxz])
    poses_bound_img.tolist()
#     print('Poses matix final...')
#     print(poses_bound_img)
    
    poses_bound.append(poses_bound_img)
    
print('FINAL')
print(poses_bound)

### Test 3: Without rot mat

In [ ]:
poses_bound = []

for i in range(1,21):
    
    #Step 1
    
    Rc = poses['Rc_'+str(i)]
    Tc = poses['Tc_'+str(i)]
    
    T_w2c = Rc
    T_w2c = np.append(T_w2c, Tc, axis = 1)

    T_w2c = np.append(T_w2c, np.transpose([[0],[0],[0],[1]]), axis = 0)
#     print(T_w2c)
    
    T_c2w = inv(T_w2c)
#     print('Inverse')
#     print(T_c2w)

    T_c2w = T_c2w[0:3,:]
#     print('Final....')
#     print(T_c2w)
    
    # NO ROTATION
    
    #Step 3
    
    rot_T_c2w = rot_T_c2w.flatten()
#     print('Flatten')
#     print(rot_T_c2w)

    poses_bound_img = np.append(rot_T_c2w, intrinsics)
#     print('Poses mat unfinished...')
#     print(poses_bound)
    
    #Step 4
    
    V_c = np.matmul(T_w2c, np.transpose(V))
#     print('Camera Verticies')
#     print(V_c)
    minz = min(V_c[2])
    maxz = max(V_c[2])
    
    poses_bound_img = np.append(poses_bound_img, [minz, maxz])
    poses_bound_img.tolist()
#     print('Poses matix final...')
#     print(poses_bound_img)
    
    poses_bound.append(poses_bound_img)
    
print('FINAL')
print(poses_bound)

### Test 4: Rot Mat 2

In [ ]:
poses_bound = []

rot_mat = np.array([[0, 1, 0, 0],
                   [-1, 0, 0, 0],
                   [0, 0, -1, 0],
                   [0, 0, 0, 1]])

for i in range(1,21):
    
    #Step 1
    
    Rc = poses['Rc_'+str(i)]
    Tc = poses['Tc_'+str(i)]
    
    T_w2c = Rc
    T_w2c = np.append(T_w2c, Tc, axis = 1)

    T_w2c = np.append(T_w2c, np.transpose([[0],[0],[0],[1]]), axis = 0)
#     print(T_w2c)
    
    #Step 2
    
    rot_T_w2c = np.matmul(T_w2c, rot_mat)
#     print('Rotated matrix....')
#     print(rot_T_w2c)
    
    #Continue Step 1

    rot_T_c2w = inv(rot_T_w2c)
#     print('Inverse')
#     print(rot_T_c2w)

    rot_T_c2w = rot_T_c2w[0:3,:]
#     print('Final....')
#     print(rot_T_c2w)
    
    #Step 3
    
    rot_T_c2w = rot_T_c2w.flatten()
#     print('Flatten')
#     print(rot_T_c2w)

    poses_bound_img = np.append(rot_T_c2w, intrinsics)
#     print('Poses mat unfinished...')
#     print(poses_bound_img)
    
    #Step 4
    
    V_c = np.matmul(rot_T_w2c, np.transpose(V))
#     print('Camera Verticies')
#     print(V_c)
    minz = min(V_c[2])
    maxz = max(V_c[2])
    
    poses_bound_img = np.append(poses_bound_img, [minz, maxz])
    poses_bound_img.tolist()
#     print('Poses matix final...')
#     print(poses_bound_img)
    
    poses_bound.append(poses_bound_img)
    
print('FINAL')
print(poses_bound)

### Test 5: Interchange height and width

In [ ]:
poses_bound = []

# [height, width, focal]
focal = (KK[0][0]+KK[1][1])/2.0

height = 612.0

width = 512.0

intrinsics = [height, width, focal]
print('Intrinsics')
print(intrinsics)

for i in range(1,21):
    
    #Step 1
    
    Rc = poses['Rc_'+str(i)]
    Tc = poses['Tc_'+str(i)]
    
    T_w2c = Rc
    T_w2c = np.append(T_w2c, Tc, axis = 1)

    T_w2c = np.append(T_w2c, np.transpose([[0],[0],[0],[1]]), axis = 0)
#     print(T_w2c)
    
    #Step 2
    
    rot_T_w2c = np.matmul(T_w2c, rot_mat)
#     print('Rotated matrix....')
#     print(rot_T_w2c)
    
    #Continue Step 1

    rot_T_c2w = inv(rot_T_w2c)
#     print('Inverse')
#     print(rot_T_c2w)

    rot_T_c2w = rot_T_c2w[0:3,:]
#     print('Final....')
#     print(rot_T_c2w)
    
    #Step 3
    
    rot_T_c2w = rot_T_c2w.flatten()
#     print('Flatten')
#     print(rot_T_c2w)

    poses_bound_img = np.append(rot_T_c2w, intrinsics)
#     print('Poses mat unfinished...')
#     print(poses_bound_img)
    
    #Step 4
    
    V_c = np.matmul(rot_T_w2c, np.transpose(V))
#     print('Camera Verticies')
#     print(V_c)
    minz = min(V_c[2])
    maxz = max(V_c[2])
    
    poses_bound_img = np.append(poses_bound_img, [minz, maxz])
    poses_bound_img.tolist()
#     print('Poses matix final...')
#     print(poses_bound_img)
    
    poses_bound.append(poses_bound_img)
    
print('FINAL')
print(poses_bound)

### Test 6: Reduce size (e-01) to match fern

In [ ]:
poses_bound = []

# [height, width, focal]
focal = (KK[0][0]+KK[1][1])/2.0

height = 0.512

width = 0.612

intrinsics = [height, width, focal]
print('Intrinsics')
print(intrinsics)

for i in range(1,21):
    
    #Step 1
    
    Rc = poses['Rc_'+str(i)]
    Tc = poses['Tc_'+str(i)]
    
    T_w2c = Rc
    T_w2c = np.append(T_w2c, Tc, axis = 1)

    T_w2c = np.append(T_w2c, np.transpose([[0],[0],[0],[1]]), axis = 0)
#     print(T_w2c)
    
    #Step 2
    
    rot_T_w2c = np.matmul(T_w2c, rot_mat)
#     print('Rotated matrix....')
#     print(rot_T_w2c)
    
    #Continue Step 1

    rot_T_c2w = inv(rot_T_w2c)
#     print('Inverse')
#     print(rot_T_c2w)

    rot_T_c2w = rot_T_c2w[0:3,:]
#     print('Final....')
#     print(rot_T_c2w)
    
    #Step 3
    
    rot_T_c2w = rot_T_c2w.flatten()
#     print('Flatten')
#     print(rot_T_c2w)

    poses_bound_img = np.append(rot_T_c2w, intrinsics)
#     print('Poses mat unfinished...')
#     print(poses_bound_img)
    
    #Step 4
    
    V_c = np.matmul(rot_T_w2c, np.transpose(V))
#     print('Camera Verticies')
#     print(V_c)
    minz = min(V_c[2])
    maxz = max(V_c[2])
    
    poses_bound_img = np.append(poses_bound_img, [minz, maxz])
    poses_bound_img.tolist()
#     print('Poses matix final...')
#     print(poses_bound_img)
    
    poses_bound.append(poses_bound_img)
    
print('FINAL')
print(poses_bound)

In [ ]:
# save to npy file
np.save('poses_bounds.npy', poses_bound)



In [ ]:
np.load('poses_bounds.npy')
